## Identifying Duplicate People/Orgs
i.e. SMG records that share the same Wikidata reference, or SMG with multiple Wikidata references. Figuring out how to resolve those duplicates

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

from collections import Counter

In [2]:
df = pd.read_pickle("../GITIGNORE_DATA/results/filtering_people_orgs_result.pkl")
df.head(2)

,LINK_ID,PREFERRED_NAME,TITLE_NAME,FIRSTMID_NAME,LASTSUFF_NAME,SUFFIX_NAME,HONORARY_SUFFIX,GENDER,BRIEF_BIO,DESCRIPTION,NOTE,BIRTH_DATE,BIRTH_PLACE,DEATH_DATE,DEATH_PLACE,CAUSE_OF_DEATH,NATIONALITY,OCCUPATION,WEBSITE,AFFILIATION,LINGUISTIC_GROUP,TYPE,REFERENCE_NUMBER,SOURCE,CREATE_DATE,UPDATE_DATE,res_ALL_NOTES,res_WIKIDATA_IDs,res_URLS,qcodes_filtered
1,10245,"Zenthon, Edward Rupert",NaN,Edward Rupert,Zenthon,NaN,NaN,M,Y,REF: http://www.iwm.org.uk/collections/item/object/1030031461,nan,1920-07,"London, Greater London, England, United Kingdom",c. 2002,NaN,NaN,British,engineer,NaN,NaN,NaN,NaN,NaN,N,28-JAN-98,05-AUG-15,REF: http://www.iwm.org.uk/collections/item/object/1030031461 --- nan,[],[http://www.iwm.org.uk/collections/item/object/1030031461],[]
2,10269,"Troughton, John",NaN,John,Troughton,NaN,NaN,M,Y,"1739 - Born in Corney, Cumbria, England; Apprenticed to his Uncle John Troughton \n1764 - traded at Surrey St., Strand, London \n1768-71 - traded at Crown Court, Fleet St., London\n1771-78 - traded at 17 Dean St., Fetter Lane, London \n1778-82 - traded at 1 Queen's Sq., Bartholomew Close, London \n1782 - purchased the business of Benjamin Cole \n1782-1788 - traded at the sign of the Orrery, 136 Fleet St, London, England. \n1788-1804 - in partnership as J & E Troughton, with brother Edward Troughton (1756-1835)","ODNB: Anita McConnell, ‘Troughton, Edward (1753–1835)’, Oxford Dictionary of National Biography, Oxford University Press, 2004; online edn, May 2005 [http://www.oxforddnb.com/view/article/27767]\nREF: A. McConnell, Instrument makers to the world: a history of Cooke, Troughton & Simms (1992) · A. W. Skempton and J. Brown, ‘John and Edward Troughton’, Notes and Records of the Royal Society, 27 (1972–3), 233–62",1739,"Broughton in Furness, Cumbria, England, United Kingdom",1807,"London, Greater London, England, United Kingdom",NaN,English; British,mathematical instrument maker,NaN,NaN,NaN,NaN,NaN,N,28-JAN-98,06-NOV-18,"1739 - Born in Corney, Cumbria, England; Apprenticed to his Uncle John Troughton \n1764 - traded at Surrey St., Strand, London \n1768-71 - traded at Crown Court, Fleet St., London\n1771-78 - traded at 17 Dean St., Fetter Lane, London \n1778-82 - traded at 1 Queen's Sq., Bartholomew Close, London \n1782 - purchased the business of Benjamin Cole \n1782-1788 - traded at the sign of the Orrery, 136 Fleet St, London, England. \n1788-1804 - in partnership as J & E Troughton, with brother Edward Troughton (1756-1835) --- ODNB: Anita McConnell, ‘Troughton, Edward (1753–1835)’, Oxford Dictionary of National Biography, Oxford University Press, 2004; online edn, May 2005 [http://www.oxforddnb.com/view/article/27767]\nREF: A. McConnell, Instrument makers to the world: a history of Cooke, Troughton & Simms (1992) · A. W. Skempton and J. Brown, ‘John and Edward Troughton’, Notes and Records of the Royal Society, 27 (1972–3), 233–62",[Q1293897],[http://www.oxforddnb.com/view/article/27767],[]


## SMG Records with multiple Wikidata references
<span style='color:green'>resolved by adding a death date filter</span>

In [3]:
df.qcodes_filtered.apply(len).value_counts()

0    14477
1     3618
Name: qcodes_filtered, dtype: int64

In [7]:
df.loc[df.qcodes_filtered.apply(len) > 1, ["LINK_ID", "PREFERRED_NAME", "BIRTH_DATE", "DEATH_DATE", "qcodes_filtered"]]

,LINK_ID,PREFERRED_NAME,BIRTH_DATE,DEATH_DATE,qcodes_filtered


## Wikidata Records with multiple SMG references

In [5]:
qcodes_count = Counter(df.qcodes_filtered.sum())
duplicate_keys = [k for k,v in qcodes_count.items() if v > 1]

In [14]:
duplicate_smg_candidates = df.loc[df['qcodes_filtered'].apply(lambda x: any([item in duplicate_keys for item in x])), ["LINK_ID", "GENDER", "PREFERRED_NAME", "BIRTH_DATE", "DEATH_DATE", "DESCRIPTION", "NOTE", "qcodes_filtered"]].sort_values('qcodes_filtered')
duplicate_smg_candidates['qcode'] = duplicate_smg_candidates['qcodes_filtered'].apply(lambda i: i[0])

duplicate_smg_candidates.to_excel("../GITIGNORE_DATA/analysis/duplicate_peopleorg_candidates.xlsx", index=False)
duplicate_smg_candidates.head()

,LINK_ID,GENDER,PREFERRED_NAME,BIRTH_DATE,DEATH_DATE,DESCRIPTION,NOTE,qcodes_filtered,qcode
6049,89234,N,Cardiff University College,NaN,NaN,http://en.wikipedia.org/wiki/Cardiff_University,"Formerly University of Wales College, Cardiff (constituent of Univerity of Wales) re-organised as Cardiff University",[Q1035745],Q1035745
4842,852,N,"Cardiff University, School of Physics & Astronomy",NaN,NaN,http://en.wikipedia.org/wiki/Cardiff_University,"Formerly University of Wales College, Cardiff (constituent of University of Wales) re-organised as Cardiff University",[Q1035745],Q1035745
9495,59374,N,Neilson and Co.,1836,1903,WIKI http://en.wikipedia.org/wiki/Neilson_and_Company,"Founded in 1836 at McAlpine Street by Walter Neilson and James Mitchell but did not begine building locomotives until 1843. The company changed name to Neilson, Reid and Company in 1898. In 1903 Neilson Reid combined with Dübs and Company and Sharp Stewart and Company to form the North British Locomotive Company",[Q1092174],Q1092174
13273,4566,N,Neilson Reid and Company,1898,1903,WIKI http://en.wikipedia.org/wiki/Neilson_and_Company,"In 1898, Neilson & Co. changed its name and the result was Neilson Reid and Co. In 1903 Neilson Reid combined with Dübs and Company and Sharp Stewart and Company to form the North British Locomotive Company.",[Q1092174],Q1092174
1226,2904,N,John Bartholomew and Son Limited,NaN,NaN,WIKI: http://en.wikipedia.org/wiki/John_Bartholomew_and_Son; CH,incorporated 1919; Company No. SC010239; dissolved 2001; name now a brandname of HarperCollins,[Q1109743],Q1109743
